<a href="https://colab.research.google.com/github/wagnerfusca/ImersaoIAAlura/blob/main/Aula05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [3]:
import google.generativeai as genai
import pandas as pd
import numpy as np

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key=api_key)

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [6]:
title = "um titulo qualquer"

sample_text = "Do mesmo modo, a complexidade dos estudos efetuados nos obriga à análise do sistema de formação de quadros que corresponde às necessidades. No mundo atual, a crescente influência da mídia estimula a padronização das regras de conduta normativas."

embeddings = genai.embed_content(model="models/embedding-001",
                                 content = sample_text,
                                 title = title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.009318376, -0.067875944, -0.019258302, 0.043202154, 0.009693939, 0.006273088, -0.051534727, -0.017456407, -0.038141914, 0.053680133, 0.016545298, 0.006556262, -0.009262777, -0.020027135, 0.014992125, 0.012207683, -0.019725062, -0.013781166, -0.014966793, -0.020918218, -0.008084611, 0.034284946, -0.020989565, -0.020444365, -0.0059982645, -0.0020174847, -0.01085688, -0.065116234, -0.0014053542, 0.03130935, -0.017551264, 0.056152068, -0.057693865, 0.0032528841, 0.019995077, -0.032850828, -0.033033255, 0.007231257, -0.020053105, 0.050842315, -0.017056972, -0.043682963, -0.010825039, 0.02526089, 0.020681836, -0.022121957, 0.022147851, 0.01947076, -0.008025303, -0.077259205, -0.008973554, -0.055783506, 0.11152829, -0.0039522215, 0.0006176549, -0.013173051, 0.0735005, -0.015834419, -0.010071987, -0.014721037, 0.0062243924, 0.0010355713, 0.031845964, 0.032170996, -0.008950626, -0.003773832, -0.032805774, 0.027572628, 0.05232061, -0.0049719466, -0.016906166, -0.014155329, 0.002

In [8]:
from contextlib import ContextDecorator
DOCUMENT1={
    "Título": "um titulo qualquer AAAA",
    "Conteúdo": "aaaaaaaa Do mesmo modo, a complexidade dos estudos efetuados nos obriga à análise do sistema de formação de quadros que corresponde às necessidades. No mundo atual, a crescente influência da mídia estimula a padronização das regras de conduta normativas."
  }

DOCUMENT2={
    "Título": "um titulo qualquer bbbb",
    "Conteúdo": "bbbb Do mesmo modo, a complexidade dos estudos efetuados nos obriga à análise do sistema de formação de quadros que corresponde às necessidades. No mundo atual, a crescente influência da mídia estimula a padronização das regras de conduta normativas."
  }

DOCUMENT3={
    "Título": "um titulo qualquer cccc",
    "Conteúdo": "cccc Do mesmo modo, a complexidade dos estudos efetuados nos obriga à análise do sistema de formação de quadros que corresponde às necessidades. No mundo atual, a crescente influência da mídia estimula a padronização das regras de conduta normativas."
  }

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [11]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,um titulo qualquer AAAA,"aaaaaaaa Do mesmo modo, a complexidade dos est..."
1,um titulo qualquer bbbb,"bbbb Do mesmo modo, a complexidade dos estudos..."
2,um titulo qualquer cccc,"cccc Do mesmo modo, a complexidade dos estudos..."


In [12]:
model = "models/embedding-001"

In [14]:
def embed_fn(title, text):
  return genai.embed_content(model= model,
                                 content = text,
                                 title = title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [17]:
df["Embutidos"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embutidos
0,um titulo qualquer AAAA,"aaaaaaaa Do mesmo modo, a complexidade dos est...","[0.035438456, -0.078063786, 0.004505374, 0.054..."
1,um titulo qualquer bbbb,"bbbb Do mesmo modo, a complexidade dos estudos...","[0.02284849, -0.0793272, -0.014737868, 0.06062..."
2,um titulo qualquer cccc,"cccc Do mesmo modo, a complexidade dos estudos...","[0.029579828, -0.06348474, -0.019741388, 0.042..."


In [18]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model= model,
                                 content = consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embutidos"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [21]:
consulta = "como tem cccc?"

gerar_e_buscar_consulta(consulta, df, model)

'cccc Do mesmo modo, a complexidade dos estudos efetuados nos obriga à análise do sistema de formação de quadros que corresponde às necessidades. No mundo atual, a crescente influência da mídia estimula a padronização das regras de conduta normativas.'